In [1]:
import geopandas as gpd

# Cargar el GeoJSON original
gdf = gpd.read_file("..\..\data\provincia\ProvinciasArgentina.geojson")

# Confirmar que las geometrías y propiedades se cargaron correctamente
print(gdf.columns)
print(gdf.head())


Index(['nombre', 'geometry'], dtype='object')
                nombre                                           geometry
0              Tucumán  POLYGON ((-66.20361 -26.62782, -66.04980 -26.6...
1  Santiago del Estero  POLYGON ((-64.42108 -26.28356, -65.03906 -27.8...
2                Salta  POLYGON ((-66.11572 -26.21459, -65.85205 -26.3...
3                Jujuy  POLYGON ((-67.22534 -23.69483, -66.47827 -24.2...
4              Formosa  POLYGON ((-62.35840 -24.08659, -58.40332 -26.8...


In [2]:
# Validar y corregir geometrías
gdf["geometry"] = gdf["geometry"].apply(lambda geom: geom if geom.is_valid else geom.buffer(0))

print("Geometrías validadas y corregidas.")


Geometrías validadas y corregidas.


In [3]:
# Guardar el GeoJSON corregido
gdf.to_file("provincias_corregido.geojson", driver="GeoJSON", encoding="utf-8")

print("GeoJSON corregido guardado como provincias_corregido.geojson.")


GeoJSON corregido guardado como provincias_corregido.geojson.


In [13]:
import pandas as pd
# Leer el CSV
sismos_df = pd.read_csv("..\..\data\sismos.csv")

# Convertir la columna 'fecha' de 'DD/MM/YYYY' a 'YYYY-MM-DD'
sismos_df['fecha'] = pd.to_datetime(sismos_df['fecha'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
# Convertir la columna 'sentido' a 1 (Sí) o 0 (No)
sismos_df['sentido'] = sismos_df['sentido'].apply(lambda x: 1 if x == 'Si' else 0)

# Verificamos el dataframe para confirmar que las fechas estén convertidas correctamente
print(sismos_df.head(12))

         fecha      hora  latitud  longitud profundidad  magnitud provincia  \
0   2024-12-02  23:28:00  -28.951   -67.303      134 Km       3.5  LA RIOJA   
1   2024-12-02  21:13:00  -29.033   -67.332      128 Km       2.5  LA RIOJA   
2   2024-12-02  20:05:00  -28.835   -68.694      119 Km       3.2  LA RIOJA   
3   2024-12-02  17:41:00  -31.749   -69.700      115 Km       2.6  SAN JUAN   
4   2024-12-02  15:57:00  -29.113   -69.431      120 Km       2.6  SAN JUAN   
5   2024-12-02  15:42:00  -24.250   -67.033      193 Km       3.6     SALTA   
6   2024-12-02  14:53:00  -23.935   -64.798       24 Km       3.3     JUJUY   
7   2024-12-02  11:44:00  -24.035   -66.803      195 Km       3.1     JUJUY   
8   2024-12-02  05:45:00  -27.796   -68.736      132 Km       2.5  LA RIOJA   
9   2024-12-02  04:29:00  -31.335   -68.718      103 Km       2.5  SAN JUAN   
10  2024-12-02  04:08:00  -31.249   -68.901      116 Km       2.5  SAN JUAN   
11  2024-12-02  02:07:00  -31.513   -68.646      108

In [14]:
import sqlite3
# Commit y cerrar la conexión
# Crear la conexión a SQLite
conn = sqlite3.connect("sismos.db")
conn.commit()
conn.close()
import os

# Ruta al archivo de la base de datos
db_file = 'sismos.db'

# Eliminar el archivo de la base de datos si existe
if os.path.exists(db_file):
    os.remove(db_file)
    print(f"La base de datos '{db_file}' ha sido eliminada.")
else:
    print(f"La base de datos '{db_file}' no existe.")


La base de datos 'sismos.db' ha sido eliminada.


In [15]:
import sqlite3
import pandas as pd

# Crear la conexión a SQLite
conn = sqlite3.connect("sismos.db")
cursor = conn.cursor()

# Crear la tabla para los datos de sismos
cursor.execute("""
CREATE TABLE IF NOT EXISTS sismos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    fecha DATE,
    hora TIME,
    latitud REAL,
    longitud REAL,
    profundidad REAL,
    magnitud REAL,
    provincia TEXT,
    sentido INTEGER
    
)
""")

# Poblar la tabla desde un archivo CSV
# Insertar los datos en la tabla
for index, row in sismos_df.iterrows():
    cursor.execute("""
    INSERT INTO sismos (fecha, hora, latitud, longitud, profundidad, magnitud, provincia, sentido)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (row['fecha'], row['hora'], row['latitud'], row['longitud'], row['profundidad'], row['magnitud'], row['provincia'], row['sentido']))

# Confirmar los cambios
conn.commit()
print("Datos insertados correctamente.")


Datos insertados correctamente.


In [17]:
cursor.execute("""
SELECT * FROM sismos 
WHERE magnitud > 3.0
""")
resultados = cursor.fetchall()
for fila in resultados:
    print(fila)


(1, '2024-12-02', '23:28:00', -28.951, -67.303, '134 Km', 3.5, 'LA RIOJA', 0)
(3, '2024-12-02', '20:05:00', -28.835, -68.694, '119 Km', 3.2, 'LA RIOJA', 0)
(6, '2024-12-02', '15:42:00', -24.25, -67.033, '193 Km', 3.6, 'SALTA', 0)
(7, '2024-12-02', '14:53:00', -23.935, -64.798, '24 Km', 3.3, 'JUJUY', 0)
(8, '2024-12-02', '11:44:00', -24.035, -66.803, '195 Km', 3.1, 'JUJUY', 0)
(12, '2024-12-02', '02:07:00', -31.513, -68.646, '108 Km', 3.9, 'SAN JUAN', 1)
(14, '2024-12-01', '22:44:00', -31.313, -69.154, '117 Km', 3.1, 'SAN JUAN', 0)
(15, '2024-12-01', '13:46:00', -24.282, -67.282, '187 Km', 3.4, 'SALTA', 0)
(19, '2024-12-01', '06:05:00', -23.675, -66.603, '246 Km', 3.5, 'JUJUY', 0)
(22, '2024-12-01', '01:33:00', -28.515, -68.318, '117 Km', 3.6, 'LA RIOJA', 1)
(23, '2024-12-01', '00:41:00', -31.529, -66.547, '135 Km', 3.4, 'LA RIOJA', 0)
(24, '2024-12-01', '00:09:00', -23.988, -66.72, '215 Km', 3.8, 'JUJUY', 0)
(25, '2024-12-01', '00:03:00', -23.324, -66.682, '219 Km', 3.4, 'JUJUY', 0)
(3

In [16]:
cursor.execute("""
SELECT * FROM sismos 
WHERE provincia = 'LA RIOJA'
""")
resultados = cursor.fetchall()
for fila in resultados:
    print(fila)


(1, '2024-12-02', '23:28:00', -28.951, -67.303, '134 Km', 3.5, 'LA RIOJA', 0)
(2, '2024-12-02', '21:13:00', -29.033, -67.332, '128 Km', 2.5, 'LA RIOJA', 0)
(3, '2024-12-02', '20:05:00', -28.835, -68.694, '119 Km', 3.2, 'LA RIOJA', 0)
(9, '2024-12-02', '05:45:00', -27.796, -68.736, '132 Km', 2.5, 'LA RIOJA', 0)
(22, '2024-12-01', '01:33:00', -28.515, -68.318, '117 Km', 3.6, 'LA RIOJA', 1)
(23, '2024-12-01', '00:41:00', -31.529, -66.547, '135 Km', 3.4, 'LA RIOJA', 0)
(38, '2024-11-29', '08:28:00', -28.394, -68.057, '126 Km', 2.5, 'LA RIOJA', 0)
(41, '2024-11-29', '03:33:00', -28.35, -68.529, '117 Km', 2.7, 'LA RIOJA', 0)
(42, '2024-11-29', '03:15:00', -29.193, -67.508, '117 Km', 2.6, 'LA RIOJA', 0)
(58, '2024-11-27', '09:06:00', -27.984, -68.997, '102 Km', 2.6, 'LA RIOJA', 0)
(87, '2024-11-25', '01:51:00', -28.928, -68.717, '107 Km', 2.8, 'LA RIOJA', 0)
(96, '2024-11-23', '23:20:00', -28.645, -68.775, '109 Km', 2.8, 'LA RIOJA', 0)
(103, '2024-11-23', '06:04:00', -29.249, -68.131, '111 Km

In [22]:
from datetime import datetime, timedelta

# Consultar los sismos del último mes desde SQLite
hoy = datetime.now()
ultimo_mes = hoy - timedelta(days=30)

query = """
SELECT latitud, longitud, magnitud, fecha 
FROM sismos 
WHERE fecha >= ?
"""
sismos_recientes = pd.read_sql_query(query, conn, params=[ultimo_mes.strftime("%Y-%m-%d")])

print(f"Sismos del último mes: {len(sismos_recientes)} registros encontrados.")


Sismos del último mes: 291 registros encontrados.
